In [ ]:
# Mounting my google drive on colab
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import 

In [ ]:
train_data = pd.read_csv("mydrive/MyDrive/Cascade Cup 2020/data/train_age_dataset.csv")
test_data = pd.read_csv("mydrive/MyDrive/Cascade Cup 2020/data/test_age_dataset.csv")

In [13]:
column_types = np.array(["r", "r", "c", "c", "n", "n", "n", "n", "n", "n", "n", "n", "n", "n", "n", "n", "n", "n", "n", "n", "n", "n", "n", "n", "n", "n", "d"])

In [10]:
column_dic = {"r": "redundant", 
              "c": "categorical",
              "n": "numeric",
              "d": "dependant"}

In [30]:
# splitting data into X's and Y's and removing redundant features
trainX = train_data.iloc[:, [x != "r" and x != "d" for x in column_types]]
trainY = train_data.iloc[:, [x == "d" for x in column_types]]
testX = test_data.iloc[:, [x != "r" and x != "d" for x in column_types[:-1]]]

# creating a concatenated dataset for normalizing or standardization
concat = pd.concat([trainX, testX], axis=0)
trainX.shape, trainY.shape, testX.shape, concat.shape

((488877, 24), (488877, 1), (54320, 24), (543197, 24))